In [1]:
import spacy
nlp = spacy.load('en')
from spacy import displacy
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
doc = nlp(u'Utah borders Idaho.')
gold = u'is_buying(Apple, U.K._startups)'

In [3]:
displacy.render(doc, jupyter=True, style='dep')

In [4]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])

(u'Utah', u'nsubj', u'borders', u'VERB', [])
(u'borders', u'ROOT', u'borders', u'VERB', [Utah, Idaho, .])
(u'Idaho', u'dobj', u'borders', u'VERB', [])
(u'.', u'punct', u'borders', u'VERB', [])


In [5]:
class NoRootFoundException(Exception):
    pass

In [6]:
def conversion(doc):
    def get_root(doc):
        for token in doc:
            if token.dep_ == 'ROOT':
                return token
        raise NoRootFoundException
    
    output = {}
    lookup = {}
    root = get_root(doc)
    output['pred'] = root.text
    lookup[root.text] = 'pred'
    
    print lookup
    
    for child in root.children:
        if child.dep_ == 'nsubj':
#             key = lookup[child.head.text]
            output['nsubj'] = child.text
#             lookup[child.text] = None
        elif child.dep_ == 'dobj':
#             key = lookup[child.head.text]
#             lookup[child.text] = None
            output['dobj'] = child.text
        else:
            key = lookup[child.head.text]
            output[key] = '_'.join([child.text, output[key]])
            lookup[output[key]] = None
    
    print output
    return '%s(%s, %s)' % (output['pred'], output['nsubj'], output['dobj'])

In [7]:
print conversion(doc)
print gold

{u'borders': 'pred'}
{'pred': u'._borders', 'dobj': u'Idaho', 'nsubj': u'Utah'}
._borders(Utah, Idaho)
is_buying(Apple, U.K._startups)


In [8]:
doc_sets = [
    u'Alice and Bob are happy.',
    u'Alice is happy, and Bob is happy.',
    u'Alice is happy. Additionally, Bob is happy.',
    u'a is a cube.',
    u'b is a cube.',
    u'a and b are cubes.',
    u'b and a are cubes.'
]

gold_sets = [
    u'cube(a)',
    u'cube(b)',
    u'cube(a) & cube(b)',
    u'cube(b) & cube(a)'
]

In [9]:
for text in doc_sets:
    doc = nlp(text)
    displacy.render(doc, jupyter=True, style='dep')

/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py:162: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", fname, loader, pkg_name)
